In [1]:
import os
import uuid

import pandas as pd
from dotenv import load_dotenv
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, PointStruct, VectorParams
from sentence_transformers import SentenceTransformer

load_dotenv()

c:\Users\S\PycharmProjects\SimilarBooksRecommendation\venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


True

In [2]:
df_1 = pd.read_csv(os.path.join(os.getcwd(), "data", "комиксы.csv"), on_bad_lines="skip").dropna()
df_2 = pd.read_csv(os.path.join(os.getcwd(), "data", "средневековье.csv"), on_bad_lines="skip").dropna()
df_3 = pd.read_csv(os.path.join(os.getcwd(), "data", "художественная.csv"), on_bad_lines="skip").dropna()

df = pd.concat([df_1, df_2, df_3], axis=0).drop_duplicates(subset=["Title", "Author"]).drop_duplicates(subset=["Title"])
df = df[df["Description"].apply(len) >= 100]

df["ID"] = [uuid.uuid5(uuid.NAMESPACE_DNS, title).hex for title in df["Title"]]


In [3]:
print(df.shape)
display(df.head())

(16149, 8)


,Title,Author,Link,Image,Category,Description,Info,ID
0,Истории книжных магазинов,Ивашкина М.,https://www.podpisnie.ru/books/istorii-knizhny...,/upload/resize_cache/iblock/cfe/160_230_1/8vg4...,Графические романы / Комиксы,Книжный магазин — место где книга и человек на...,Автор Ивашкина М. Издательство Миля Год издани...,e26498c9c4a75252a18a1c677b6d3ae9
1,Полуночники,Эвенс Б.,https://www.podpisnie.ru/books/polunochniki/,/upload/resize_cache/iblock/a5b/160_230_1/tj5v...,Графические романы / Комиксы,Одна ночь. Три незнакомца. Три совершенно разн...,Автор Эвенс Б. Издательство Бумкнига Год издан...,2d4a036599e959a0a9e76e32a2075200
2,Госпожа Кагуя: В любви как на войне. Любовная...,Акасака А.,https://www.podpisnie.ru/books/gospozha-kaguya...,/upload/resize_cache/iblock/7fd/160_230_1/uqfn...,Графические романы / Комиксы,Завещание Ганъана Синомии уничтожено и Кагуя в...,Автор Акасака А. Издательство Азбука Год издан...,9ac06e3bafd25975bbf6fb9083e18567
3,"Люди, которые легко становятся счастливыми. ...",Дэнсинг С.,https://www.podpisnie.ru/books/lyudi-kotorye-l...,/upload/resize_cache/iblock/929/160_230_1/y4yj...,Графические романы / Комиксы,Привычная рутина отбирает краски жизни а о «ма...,Автор Дэнсинг С. Издательство КоЛибри Год изда...,ca68d65336425ee49bb1e563fb832e1f
4,"Моя геройская академия. Кн. 19. Те, кто об...",Хорикоси К.,https://www.podpisnie.ru/books/moya-geroyskaya...,/upload/resize_cache/iblock/f6d/160_230_1/7hh3...,Графические романы / Комиксы,Решающее сражение за судьбу мира в самом разга...,Автор Хорикоси К. Издательство Азбука Год изда...,2cb378c590605ddb8c4cf85b8ee1518f


In [4]:
embedder = SentenceTransformer(os.getenv("QDRANT_EMBEDDER"))
qdrant = QdrantClient(url=os.getenv("QDRANT_URL"), api_key=os.getenv("QDRANT_API_KEY"))


In [5]:
vectors = embedder.encode(df["Description"].values, batch_size=1, normalize_embeddings=True, show_progress_bar=True)

Batches:   0%|          | 0/16149 [00:00<?, ?it/s]

In [6]:
qdrant.recreate_collection(
    collection_name=os.getenv("QDRANT_COLLECTION_NAME"),
    vectors_config=VectorParams(size=312, distance=Distance.COSINE),
)


C:\Users\S\AppData\Local\Temp\ipykernel_36236\3353009504.py:1: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


True

In [7]:
from tqdm import tqdm

# Параметры для пакетной обработки
batch_size = 300  # Размер пакета, можно настроить по мере необходимости
points = []

# Обрабатываем данные по пакетам
for uid, category, author, title, image_link, vector in tqdm(
    zip(df["ID"], df["Category"], df["Author"], df["Title"], df["Image"], vectors)
):
    points.append(
        PointStruct(
            id=uid,
            payload={
                "category": category,
                "author": author,
                "title": title,
                "image_link": image_link,
            },
            vector=vector,
        )
    )

    # Если пакет набран, отправляем его в Qdrant
    if len(points) >= batch_size:
        qdrant.upsert(collection_name=os.getenv("QDRANT_COLLECTION_NAME"), points=points)
        points = []  # Очищаем список для следующего пакета

# Отправляем оставшиеся точки, если они есть
if points:
    qdrant.upsert(collection_name=os.getenv("QDRANT_COLLECTION_NAME"), points=points)


16149it [00:39, 405.12it/s]


In [10]:
query_vector = embedder.encode("Ведьмак Предназначение")
n_chunks = 5
offset = 0


In [11]:
message_content = qdrant.search(
    collection_name=os.getenv("QDRANT_COLLECTION_NAME"), query_vector=query_vector, limit=n_chunks, offset=offset
)
message_content


[ScoredPoint(id='7654c1a4-09e6-5ff8-817d-b43929fa6d87', version=39, score=0.8305099, payload={'author': 'Сапковский А.', 'category': 'Художественная литература', 'image_link': '/upload/resize_cache/iblock/fb9/160_230_1/6tjf0vcl6qf92zgz17ku1t1ewba720o8.jpg', 'title': 'Ведьмак.  Меч Предназначения'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id='92444711-200a-5c29-aa4a-892effbf7a05', version=41, score=0.82693565, payload={'author': 'Сапковский А.', 'category': 'Художественная литература', 'image_link': '/upload/resize_cache/iblock/c53/160_230_1/tigq1chxqc0e3yev85p31n39xaly6h3f.jpg', 'title': 'Ведьмак.  Последнее желание'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id='de468708-c02d-5054-b28c-d3284c5bd3c5', version=40, score=0.8225369, payload={'author': 'Сапковский А.', 'category': 'Художественная литература', 'image_link': '/upload/resize_cache/iblock/c71/160_230_1/e24vw9bjg1l753zw24w2du49pzh9e86q.jpg', 'title': 'Ведьмак.  Час Презрения (2изд)'}, ve

# Redis

In [18]:
import redis

# Подключение к Redis
redis_client = redis.StrictRedis(
    host=os.getenv("REDIS_HOST"), port=os.getenv("REDIS_PORT"), password=os.getenv("RESID_PASSWORD")
)

try:
    # Проверка подключения к Redis
    pong = redis_client.ping()
    if pong:
        print("Подключение к Redis успешно!")
    else:
        print("Не удалось подключиться к Redis.")
except redis.ConnectionError as e:
    print(f"Ошибка подключения к Redis: {e}")


Подключение к Redis успешно!


## Быстрая загрузка данных в redis

In [20]:
# from tqdm import tqdm

# # Преобразование данных в список кортежей для удобства использования
# data = list(zip(df["Title"], df["Description"], df["Category"], df["Author"]))
# # Определение размера батча
# batch_size = 1000

# # Использование пайплайна для записи данных и метаданных
# for i in tqdm(range(0, len(data), batch_size)):
#     with redis_client.pipeline() as pipe:
#         for title, description, category, author in data[i:i + batch_size]:
#             # Установка данных
#             pipe.set(title, description)
            
#             # Установка метаданных
#             metadata_key = f"{title}:metadata"
#             pipe.hset(metadata_key, "category", category)
#             pipe.hset(metadata_key, "author", author)
        
#         # Выполнение всех команд в пайплайне
#         pipe.execute()

100%|██████████| 17/17 [00:12<00:00,  1.38it/s]


In [55]:
num_keys = redis_client.dbsize()

print(f"Number of keys in the database: {num_keys}")

Number of keys in the database: 32298


# Гибридный поиск

In [28]:
from typing import Dict, Optional

def fetch_data_and_metadata(redis_connection: redis.StrictRedis, name: str) -> Dict[str, Optional[str]]:
    """
    Получает данные и метаданные из Redis по ключу `name`.
    
    :param redis_connection: Подключение к Redis.
    :param name: Ключ для извлечения данных и метаданных.
    :return: Словарь с данными и метаданными.
    """
    # Формирование ключа для метаданных
    metadata_key = f"{name}:metadata"
    
    # Использование пайплайна для извлечения данных и метаданных
    with redis_connection.pipeline() as pipe:
        # Запрос данных
        pipe.get(name)
        # Запрос метаданных
        pipe.hgetall(metadata_key)
        
        # Выполнение всех команд в пайплайне
        responses = pipe.execute()
    
    # Извлечение данных и метаданных из ответов
    description = responses[0]
    metadata = responses[1]
    
    # Декодирование данных, если они существуют
    if description is not None:
        description = description.decode('utf-8')  # Декодирование байтов в строку
    
    # Декодирование метаданных, если они существуют
    if metadata is not None:
        metadata = {k.decode('utf-8'): v.decode('utf-8') for k, v in metadata.items()}
    else:
        metadata = {}
    
    # Формирование итогового результата
    result = {
        'description': description,
        'metadata': metadata
    }
    
    return result

In [73]:
from qdrant_client.http import models

def hybrid_search(query, category_filter=None):
    # Создание векторного запроса
    query_vector = embedder.encode(query)
    

    # Построение условий фильтрации
    filter_conditions = []
    if category_filter:
        filter_conditions.append(models.FieldCondition(
            key='category',
            match=models.MatchValue(value=category_filter)
        ))

    # Выполнение поиска
    results = qdrant.search(
        collection_name=os.getenv("QDRANT_COLLECTION_NAME"),
        query_vector=query_vector,
        query_filter=models.Filter(
            must=filter_conditions
        ) if filter_conditions else None,
        limit=10
    )

    return results


In [74]:
query = "Полуночники"

# Получение данных и метаданных
data_and_metadata = fetch_data_and_metadata(redis_client, name)
description = data_and_metadata['description']
category = data_and_metadata['metadata']["category"]
author = data_and_metadata['metadata']["author"]

In [80]:
# Пример запроса
search_results = hybrid_search(
    query=description,
    # category_filter=category,
    # author_filter=author,
    # title_filter=query
)

for result in search_results:
    print(f"Title: {result.payload['title']}, Score: {result.score}, Category: {result.payload['category']}")

Title: Полуночники, Score: 1.0000001, Category: Графические романы / Комиксы
Title: Стигматы, Score: 0.90010583, Category: Графические романы / Комиксы
Title: Лучшая фантастика, Score: 0.8936212, Category: Художественная литература
Title: Создания ночи, Score: 0.89227223, Category: Графические романы / Комиксы
Title: Мунк, Score: 0.89163905, Category: Графические романы / Комиксы
Title: Джонатан Стрендж и мистер Норрелл, Score: 0.89122635, Category: Художественная литература
Title: Ангелы и насекомые, Score: 0.8911444, Category: Художественная литература
Title: Маркиз Кит де ла Бален, Score: 0.8909486, Category: Графические романы / Комиксы
Title: Огненный шар, Score: 0.88734776, Category: Графические романы / Комиксы
Title: Мистические рассказы(По Э.  А.  ), Score: 0.8871362, Category: Художественная литература
